In [1]:
import os
import pandas as pd
from tqdm.notebook import tqdm
tqdm().pandas()
import numpy as np

import itertools as it
from collections import Counter

from sklearn.metrics.pairwise import cosine_similarity

path = 'D:/Users/Nicola Melluso/ScientificPublications/'
path_vectors = path + 'Papers/TitleAbstract/papers_vectors/'

path_processed = path + 'PapersProcessed/'

gram = 'unigrams'
gram_label = gram + '_comb'

0it [00:00, ?it/s]

## ************** 

## Preliminaries

### 1. Papers to be done

In [2]:
reader = open('./data/paperIds.txt','r')
paperIds = set()

for line in tqdm(reader,total = 72245396):
    paperIds.add(int(line.replace('\n','')))
    

reader = open('./data/paperIdsNov.txt','r')
paperIdsNov = set()

for line in tqdm(reader,total = 33536292):
    paperIdsNov.add(int(line.replace('\n','')))

  0%|          | 0/72245396 [00:00<?, ?it/s]

  0%|          | 0/33536292 [00:00<?, ?it/s]

### 2. Gram to idx 

In [5]:
reader = open('./data/%s_to_idx.csv'%(gram),'r', encoding="utf-8")
reader.__next__()

gram_to_idx = {}
idx_to_gram = {}

for line in tqdm(reader, total = 7363821):
    
    if ',' not in line:
        continue
        
    line = line.split(',')
    try:
        gram_to_idx[line[0]] = int(line[1].replace('\n',''))
        idx_to_gram[int(line[1].replace('\n',''))] = line[0]
    except Exception:
        continue
        
del gram_to_idx['']

  0%|          | 0/7363821 [00:00<?, ?it/s]

### 3. Core analysis

In [13]:
print('Reading baseline...')
baseline = set()

reader = open(path_processed + '%s_baseline.txt'%(gram_label),'r', encoding="utf-8")

for line in tqdm(reader, total = 10809447):
    
    baseline.add(tuple(sorted([int(i) for i in line.replace('\n','').split(',')])))

# remove empty space combinations
baseline = set([i for i in tqdm(baseline) if 1 not in i])
        
print('Analysis...')

reader = open('./data/%s.csv'%(gram.split('_')[0]),'r', encoding="utf-8")
reader.__next__()

writer = open('./data/new_%s/new_%s_paperId.csv'%(gram_label,gram_label),'w', encoding="utf-8")
writer.write('gram1_idx,gram2_idx,PaperId\n')

counter = Counter()

for line in tqdm(reader, total = 103061513):
    
    line = line.split('\t')
    
    paperId = int(line[0])
    
    if paperId not in paperIds:
        continue
                
    text = set(' '.join(line[1:]).split(' '))
    text = [gram_to_idx.get(t) for t in text]
    text = set([t for t in text if t is not None])

    combs = list(it.combinations(text,2))

    combs = [c for c in combs if c not in baseline]
    
    for comb in combs:
        comb = tuple(sorted(comb))
        if comb not in counter:
            if paperId not in paperIdsNov:
                continue
            
            counter[comb] = 0
            
            ## write the PaperId
            writer.write(','.join([
                str(i) for i in [
                    comb[0],
                    comb[1],
                    paperId
                ]
            ]) + '\n')
        else:
            counter[comb] += 1
            
writer.close()
    
print('Exporting the counter...')
writer = open('./data/new_%s/new_%s_reuse.csv'%(gram_label,gram_label),'w', encoding="utf-8")
writer.write('gram1_idx,gram2_idx,reuse\n')

for comb,reuse in tqdm(counter.items(), total = len(counter)):
    writer.write(','.join([
                str(i) for i in [
                    comb[0],
                    comb[1],
                    reuse
                ]
            ]) + '\n')
    
writer.close()

Reading baseline...


  0%|          | 0/10809447 [00:00<?, ?it/s]

  0%|          | 0/10172374 [00:00<?, ?it/s]

Analysis...


  0%|          | 0/103061513 [00:00<?, ?it/s]

Exporting the counter...


  0%|          | 0/2579272666 [00:00<?, ?it/s]

## 2. Adjust non-doi and citations minimum count 

#### Read grams with only reuse > 5

In [4]:
#counter = set()
combinations = {}

reader = open('./data/new_%s/new_%s_reuse.csv'%(gram_label,gram_label),'r', encoding="utf-8")
reader.__next__()

for line in tqdm(reader, total = 2579272666):
    line = line.split(',')
    
    reuse = int(line[2].replace('\n',''))
    
    if reuse <= 5:
        continue
        
    comb = (int(line[0]),int(line[1]))
    
    combinations[comb] = np.nan


  0%|          | 0/2579272666 [00:00<?, ?it/s]

### Re-do analysis 

Create a new file `new_gram_PaperId_DC.csv` with the real PaperIds.

In [14]:
print('Reading baseline...')
baseline = set()
dones = set()
counter = Counter()

reader = open(path_processed + '%s_baseline.txt'%(gram_label),'r', encoding="utf-8")

for line in tqdm(reader, total = 10809447):
    
    baseline.add(tuple(sorted([int(i) for i in line.replace('\n','').split(',')])))

# remove empty space combinations
baseline = set([i for i in tqdm(baseline) if 1 not in i])

print('Analysis...')

reader = open('./data/%s.csv'%(gram.split('_')[0]),'r', encoding="utf-8")
reader.__next__()

writer = open('./data/new_%s/new_%s_paperId_DC.csv'%(gram_label,gram_label),'w', encoding="utf-8")
writer.write('gram1_idx,gram2_idx,PaperId\n')


for line in tqdm(reader, total = 103117245):
    
    line = line.split('\t')
    
    paperId = int(line[0])
    
    if paperId not in paperIds:
        continue
                
    text = set(' '.join(line[1:]).split(' '))
    text = [gram_to_idx.get(t) for t in text]
    text = [t for t in text if t is not None]

    combs = list(it.combinations(text,2))

    combs = [c for c in combs if c not in baseline]
    
    for comb in combs:
        
        comb = tuple(sorted(comb))
        
        if comb not in combinations:
            if comb not in counter:
            
                ## write the PaperId real
                writer.write(','.join([
                    str(i) for i in [
                        comb[0],
                        comb[1],
                        paperId
                    ]
                ]) + '\n')
                
                counter[comb] = 0
            else:
                counter[comb] += 1
                
writer.close()
reader.close()

print('Exporting the counter...')
writer = open('./data/new_%s/new_%s_reuse_DC.csv'%(gram_label,gram_label),'w', encoding="utf-8")
writer.write('gram1_idx,gram2_idx,reuse\n')

for comb,reuse in tqdm(counter.items(), total = len(counter)):
    writer.write(','.join([
                str(i) for i in [
                    comb[0],
                    comb[1],
                    reuse
                ]
            ]) + '\n')
    
writer.close()


Reading baseline...


  0%|          | 0/10809447 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
print('Exporting the counter...')
writer = open('./data/new_%s/new_%s_reuse_DC.csv'%(gram_label,gram_label),'w', encoding="utf-8")
writer.write('gram1_idx,gram2_idx,reuse\n')

for comb,reuse in tqdm(counter.items(), total = len(counter)):
    writer.write(','.join([
                str(i) for i in [
                    comb[0],
                    comb[1],
                    reuse
                ]
            ]) + '\n')
    
writer.close()

Exporting the counter...


  0%|          | 0/2863311530 [00:00<?, ?it/s]

# ********* Re-start kernel ********** 

# Aggregations 
### To execute after the counter

## Merging new word combs 

In [20]:
print('Reading papers...')
papers = pd.read_csv('./data/PapersData.csv', converters = {'PaperId':int}, usecols = ['PaperId'])

stop_index = papers[papers['PaperId'] == 2221882060].index[0]
print('Creating papersId set...')
paperIds = set(papers[stop_index:]['PaperId'].unique())

In [23]:
gram_label = 'unigrams_comb'
writer = open('./data/new_%s/new_%s_gram_idx.csv'%(gram_label,gram_label), 'w')
writer.write('gram_idx_1,gram_idx_2,PaperId,reuse\n')

combs_reuse = open('./data/new_%s/new_%s_reuse.csv'%(gram_label,gram_label),  encoding="utf-8")
combs_paperId = open('./data/new_%s/new_%s_PaperId.csv'%(gram_label,gram_label),  encoding="utf-8")

combs_reuse.__next__()
combs_paperId.__next__()

i = 0

for line_reuse,line_paperId in tqdm(zip(combs_reuse, combs_paperId)):
    
    i += 1
    
    line_reuse = line_reuse.replace('\n','').split(',')
    reuse = int(line_reuse[-1])
    paperId = int(line_reuse[2])
    
    
    ## random check
    if i % 1000000 == 0:
        assert line_reuse[0] == line_paperId.split(',')[0]
        assert line_reuse[1] == line_paperId.split(',')[1]
    
    
    if reuse <= 5:
        if paperId in paperIds:
            writer.write(line_paperId.replace('\n',',') + str(reuse) + '\n')
            continue
        else:
            continue
        
    writer.write(line_paperId.replace('\n',',') + str(reuse) + '\n')

0it [00:00, ?it/s]

In [25]:
gram_label = 'unigrams_comb'
writer = open('./data/new_%s/new_%s_gram_idx.csv'%(gram_label,gram_label), 'a')
#writer.write('gram_idx_1,gram_idx_2,PaperId,reuse\n')

combs_reuse = open('./data/new_%s/new_%s_reuse_DC.csv'%(gram_label,gram_label),  encoding="utf-8")
combs_paperId = open('./data/new_%s/new_%s_PaperId_DC.csv'%(gram_label,gram_label),  encoding="utf-8")

combs_reuse.__next__()
combs_paperId.__next__()

i = 0

for line_reuse,line_paperId in tqdm(zip(combs_reuse, combs_paperId), total = 2863311530):
    
    i += 1
    
    line_reuse = line_reuse.replace('\n','').split(',')
    reuse = int(line_reuse[-1])
    paperId = int(line_reuse[2])
    
    
    ## random check
    if i % 1000000 == 0:
        assert line_reuse[0] == line_paperId.split(',')[0]
        assert line_reuse[1] == line_paperId.split(',')[1]
    
    
    if reuse == 0:
        continue
        
    writer.write(line_paperId.replace('\n',',') + str(reuse) + '\n')
    
writer.close()

  0%|          | 0/2863311530 [00:00<?, ?it/s]

#### Gram to idx for word combs 

In [30]:
writer = open('./data/new_%s/new_%s.csv'%(gram_label,gram_label), 'w')
writer.write('word1,word2,PaperId,reuse\n')

reader = open('./data/new_%s/new_%s_gram_idx.csv'%(gram_label,gram_label), 'r')
reader.__next__()

for line in tqdm(reader):
    line = line.split(',')
    line[0] = idx_to_gram[int(line[0])]
    line[1] = idx_to_gram[int(line[1])]
    
    writer.write(','.join(line))
    
writer.close()



0it [00:00, ?it/s]

Chunking strategy 

In [2]:
reader = open('./data/%s_to_idx.csv'%(gram),'r', encoding="utf-8")
reader.__next__()

idx_to_gram = {}

for line in tqdm(reader, total = 20880952):
    
    if ',' not in line:
        continue
        
    line = line.split(',')
#    idx_to_gram[int(line[1].replace('\n',''))] = line[0]
    
    try:
        idx_to_gram[line[1]] = int(line[0].replace('\n',''))
    except Exception:
        continue

  0%|          | 0/20880952 [00:00<?, ?it/s]

# old

### Adjust for fixing the issue wiith memory
There are combinations that appear in all papers (non-DOI and citations < 5) and are re-used less than 5 times that are not counted for papers after 2015 (specifically after the paperId 2221882060).
For combinations that appear after 2015 with re-use lower than 5, consider only the ones that come from papers with DOI.

> 5 or in PaperIds > of the order  2221882060

#### PaperId sorted by date: set of patents after the 2221882060

### Stopping PaperId 

In [19]:
paperId_stop = 2221882060

2221882060

In [2]:
papers = pd.read_csv('./data/PapersData.csv', converters = {'PaperId':int}, usecols = ['PaperId'])
papers

,PaperId
0,1967467172
1,2034577926
2,2070022537
3,3139928427
4,2073390270
...,...
72245391,3113519567
72245392,3117632671
72245393,2934725299
72245394,3090313612


In [5]:
papers = pd.read_csv('./data/PapersData.csv', converters = {'PaperId':int}, usecols = ['PaperId'])

stop_index = papers[papers['PaperId'] == 2221882060].index[0]
paperIds = set(papers[stop_index:]['PaperId'].unique())

In [3]:
chunksize = 1000000000

ds_paperId = pd.read_csv('./data/new_%s/new_%s_paperId.csv'%(gram_label,gram_label), chunksize = chunksize, converters = {'PaperId':int,'gram1_idx':int, 'gram2_idx':int},  encoding="utf-8")


writer = open('./data/new_%s/new_%s_2.csv'%(gram_label,gram_label),'w')
writer.write('word1,word2,PaperId,reuse\n')
writer.close()



print('First iterate over all...')
# iterate over paperIds
for df_paperId in tqdm(ds_paperId, total = round(2579272666/chunksize)):
    
    reuses_dones = [-1]
    
    to_do_ids = set(list(df_paperId['PaperId'].unique()))
    
    if to_do_ids == set():
        continue

    ds_reuse = pd.read_csv('./data/new_%s/new_%s_reuse.csv'%(gram_label,gram_label), chunksize = chunksize, converters = {'PaperId':int,'gram1_idx':int, 'gram2_idx':int},  encoding="utf-8")
        
    for i_reuse,df_reuse in tqdm(enumerate(ds_reuse)):
        
        if i_reuse <= max(reuses_dones):
            continue
        
        #df_reuse = df_reuse[(df_reuse['reuse'] > 5) | (df_reuse['reuse'].isin(paperIds))]
        
        df = pd.merge(df_paperId,df_reuse, on = ['gram1_idx','gram2_idx'])
        
        dones_ids = set(df['PaperId'].unique())
        
        df = df[(df['reuse'] > 5) | (df['reuse'].isin(paperIds))]
                    
        if df.empty:
            continue
            

        df['word1'] = df['gram1_idx'].apply(lambda x: idx_to_gram.get(x))
        df['word2'] = df['gram2_idx'].apply(lambda x: idx_to_gram.get(x))

        df = df[['word1','word2','PaperId','reuse']]

        df.to_csv('./data/new_%s/new_%s.csv'%(gram_label,gram_label), mode = 'a', header = False, index = False)
        
        
        to_do_ids = to_do_ids - dones_ids

        if to_do_ids == set():
            reuses_dones.append(i_reuse)
            break
    
print('Second iterate over non-DOI and citation minimum...')

ds_paperId_DC = pd.read_csv('./data/new_%s/new_%s_paperId_DC.csv'%(gram_label,gram_label), chunksize = chunksize, converters = {'PaperId':int,'gram1_idx':int, 'gram2_idx':int},  encoding="utf-8")

for df_paperId_DC in tqdm(ds_paperId_DC, total = round(2863311530/chunksize)):
    
    reuses_dones = [-1]
    
    to_do_ids = set(list(df_paperId_DC['PaperId'].unique()))
    
    if to_do_ids == set():
        continue
    
    ds_reuse = pd.read_csv('./data/new_%s/new_%s_reuse_DC.csv'%(gram,gram), chunksize = chunksize, converters = {'PaperId':int,'gram1_idx':int, 'gram2_idx':int}, encoding="utf-8")
    
    for i_reuse, df_reuse in tqdm(enumerate(ds_reuse)):
        
        if i_reuse <= max(reuses_dones):
            continue

        
        df = pd.merge(df_paperId_DC,df_reuse, on = ['gram1_idx','gram2_idx'])
        
        dones_ids = set(df['PaperId'].unique())
        
        df = df[df['reuse'] > 0]
        
        if df.empty:
            continue
        
        df['word1'] = df['gram1_idx'].apply(lambda x: idx_to_gram.get(x))
        df['word2'] = df['gram2_idx'].apply(lambda x: idx_to_gram.get(x))
        
        df = df[['word1','word2','PaperId','reuse']]
        
        df.to_csv('./data/new_%s/new_%s.csv'%(gram_label,gram_label), mode = 'a', header = False, index = False)     
        
        
        to_do_ids = to_do_ids - dones_ids
        
        if to_do_ids == set():
            reuses_dones.append(i_reuse)
            break



First iterate over all...


  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

NameError: name 'paperIds' is not defined

### Large memory batch 
#### 1st iteration

In [ ]:
print('Reading PaperIds...')
papers = pd.read_csv('./data/PapersData.csv', converters = {'PaperId':int}, usecols = ['PaperId'])

stop_index = papers[papers['PaperId'] == 2221882060].index[0]
paperIds = set(papers[stop_index:]['PaperId'].unique())

print('Reading combs reuse...')
df_reuse = pd.read_csv('./data/new_%s/new_%s_reuse.csv'%(gram_label,gram_label), converters={'PaperId': int, 'gram1_idx':int, 'gram2_idx':int, 'reuse': int},  encoding="utf-8")
df_reuse = df_reuse[df_reuse['reuse'] > 5]

# Define the filtering condition for "PaperId"
filter_condition = lambda x: int(x) if int(x) not in paperIds else 0

print('Reading combs paperId...')
df_paperId = pd.read_csv('./data/new_%s/new_%s_paperId.csv'%(gram_label,gram_label), converters = {'PaperId':filter_condition,'gram1_idx':int, 'gram2_idx':int},  encoding="utf-8")
print('Converting PaperId...')
df_paperId = df_paperId[df_paperId['PaperId'] != 0]

print('Merging...')
df = pd.merge(df_paperId,df_reuse, on = ['gram1_idx','gram2_idx'])

print('Converting grams...')
df['word1'] = df['gram1_idx'].apply(lambda x: idx_to_gram.get(x))
df['word2'] = df['gram2_idx'].apply(lambda x: idx_to_gram.get(x))

df = df[['word1','word2','PaperId','reuse']]

print('Exporting...')
df.to_csv('./data/new_%s/new_%s_2.csv'%(gram_label,gram_label), index = False)



Reading PaperIds...
Reading combs reuse...
Reading combs paperId...
Converting PaperId...
Merging...


## ---- restart kernel ----- 

#### 2nd iteration 

In [ ]:
print('Reading combs reuse...')
df_reuse = pd.read_csv('./data/new_%s/new_%s_reuse_DC.csv'%(gram_label,gram_label), converters={'PaperId': int, 'gram1_idx':int, 'gram2_idx':int, 'reuse': int},  encoding="utf-8")
df_reuse = df_reuse[df_reuse['reuse'] > 0]

print('Reading combs paperId...')
df_paperId = pd.read_csv('./data/new_%s/new_%s_paperId_DC.csv'%(gram_label,gram_label), converters = {'PaperId':int,'gram1_idx':int, 'gram2_idx':int},  encoding="utf-8")
print('Converting PaperId...')

print('Merging...')
df = pd.merge(df_paperId,df_reuse, on = ['gram1_idx','gram2_idx'])

print('Converting grams...')
df['word1'] = df['gram1_idx'].apply(lambda x: idx_to_gram.get(x))
df['word2'] = df['gram2_idx'].apply(lambda x: idx_to_gram.get(x))

df = df[['word1','word2','PaperId','reuse']]

print('Exporting...')
df.to_csv('./data/new_%s/new_%s_2.csv'%(gram_label,gram_label), mode = 'a', header = False, index = False)


## ******* Preliminary ********* 

## 2B. Define baseline 

In [ ]:
years = range(1880,1901)

baseline = set()
baseline_title = set()

for year in tqdm(years, total = len(years)):
    df = pd.read_parquet(path_processed + gram + '/%d.parquet'%(year))
    
    df = df[df['Flag'] == 'Green'].fillna('')
    
    df[gram] = df[gram].apply(lambda x: [gram_to_idx.get(t) for t in list(set(x.split(' ')))])
    df[gram] = df[gram].apply(lambda x: set([t for t in x if t is not None]))
    
    df[gram + '_title'] = df[gram + '_title'].apply(lambda x: [gram_to_idx.get(t) for t in list(set(x.split(' ')))])
    df[gram + '_title'] = df[gram + '_title'].apply(lambda x: set([t for t in x if t is not None]))
    
    
    for _,row in df.iterrows():
        title_abstract = row[gram + '_title'] | row[gram]
        title = row[gram + '_title']
        
        title_abstract_combs = set(list(it.combinations(title_abstract,2)))
        title_combs = set(list(it.combinations(title,2)))
                    
        baseline.update(title_abstract_combs)
        baseline_title.update(title_combs)
    

print('Exporting title...')

writer = open(path_processed + '%s_title_baseline.txt'%(gram_label),'w',  encoding="utf-8")

for comb in tqdm(baseline_title, total = len(baseline_title)):
    writer.write(str(comb[0]) + ',' + str(comb[1]) + '\n')
    
writer.close()

print('Exporting title + abstract...')

writer = open(path_processed + '%s_baseline.txt'%(gram_label),'w',  encoding="utf-8")

for comb in tqdm(baseline, total = len(baseline)):
    writer.write(str(comb[0]) + ',' + str(comb[1]) + '\n')
    
writer.close()